In [1]:
from collections import namedtuple
import requests
from bs4 import BeautifulSoup
import time
import re

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
class Team:
      def __init__(self, hit, homerun, steal, strikeout, doubleplay, error):
            self.hit = hit
            self.homerun = homerun
            self.steal = steal
            self.strikeout = strikeout
            self.doubleplay = doubleplay
            self.error = error

      def get_scores(self):
            return [
                  self.hit,         # Score 1
                  self.homerun,     # Score 2
                  self.steal,       # Score 3
                  self.strikeout,   # Score 4
                  self.doubleplay,  # Score 5
                  self.error        # Score 6
            ]

In [4]:
# Set up ChromeDriver
service = Service(ChromeDriverManager().install())

In [5]:
driver = webdriver.Chrome(service=service)

# URL of the webpage to scrape
url = 'https://m.sports.naver.com/game/20240613LGSS02024/record'

# Open the webpage
driver.get(url)

# Optionally wait for the elements to load (adjust the time as needed)
driver.implicitly_wait(10)

In [6]:
try:
    # Find all div elements with the specified class
    score_divs = driver.find_elements(By.CLASS_NAME, 'TeamVS_score__2Iv0U')
    
    # Iterate over the found elements and print their text content
    for index, score_div in enumerate(score_divs, start=1):
        score = score_div.text
        print(f'Score {index}: {score}')
except Exception as e:
    print(f'Error: {e}')
finally:
    driver.close()

Score 1: 5
Score 2: 8
Score 3: 0
Score 4: 3
Score 5: 0
Score 6: 0
Score 7: 3
Score 8: 5
Score 9: 0
Score 10: 0
Score 11: 1
Score 12: 0


In [7]:
Team1 = Team(hit=score_divs[0], homerun=score_divs[2], steal=score_divs[4], strikeout=score_divs[6], doubleplay=score_divs[8], error=score_divs[10])
Team2 = Team(hit=score_divs[1], homerun=score_divs[3], steal=score_divs[5], strikeout=score_divs[7], doubleplay=score_divs[9], error=score_divs[11])

#content > div > div.Home_main_section__y9jR4 > section.Home_game_panel__97L_8 > div.Home_game_contents__35IMT > div > div.PlayerRecord_comp_player_record__1tI5G.type_kbo > div.PlayerRecord_tabpanel__3GYt9 > div:nth-child(1) > div.PlayerRecord_player_list_area__gbho3 > ul > li:nth-child(1) > a

In [8]:
html = requests.get(url).text
soup = BeautifulSoup(html, "html5lib")

In [9]:
name = soup.find_all('span', class_ = 'PlayerRecord_name__1W_c0')
print(name)

[]


In [10]:
def remove_newlines(text):
    return text.replace("\n", "").replace("\r", "")

In [11]:
driver = webdriver.Chrome(service=service)

# Open the webpage
driver.get(url)

# Optionally wait for the elements to load (adjust the time as needed)
driver.implicitly_wait(10)

In [12]:
print(f'타순\t포지션\t이름')
try:
    # Find all div elements with the specified class
    name_elements = driver.find_elements(By.CLASS_NAME, 'PlayerRecord_name__1W_c0')
    position_elements = driver.find_elements(By.CLASS_NAME, 'PlayerRecord_position__3SBbd')
    order = 0
    
    # Iterate over the found elements and print their text content
    for index, (name_elem, pos_elem) in enumerate(zip(name_elements, position_elements), start=1):
        name = name_elem.text.strip()  # strip()을 이용해 공백 제거
        pos = pos_elem.text.strip()
        
        if name:
            if pos == "대타":
                print(f'교체\t{pos}\t{name}')  # 공백을 추가하여 올바르게 출력
            else:
                order = order + 1                           # strip()을 이용해 공백 제거
                print(f'{order}번\t{pos}\t{name}')     # 공백을 추가하여 올바르게 출력
                              
except Exception as e:
    print(f'Error: {e}')
finally:
    driver.close()

타순	포지션	이름
1번	중견수	김지찬
2번	유격수	이재현
3번	지명타자	구자욱
4번	1루수	박병호
5번	3루수	김영웅
6번	포수	강민호
교체	대타	이병헌
7번	우익수	이성규
8번	2루수	김동진
9번	좌익수	윤정빈


실시간 중계 data 읽기

In [13]:
class RelayList_result:
      main_info: str
      winning_rate: float
      winning_rate_diff: float

class RelayList_history(RelayList_result):
      ball_cnt: int

class RelayList_pitch(RelayList_history):
      type: str
      speed: int
      stuff: str
      

In [14]:
# 중계 탭 url 읽기
url = 'https://m.sports.naver.com/game/20240613LGSS02024/relay'

In [15]:
# 특정 클래스의 하위 요소 찾기
relay_player_area = 'RelayList_player_area__2ur0q'
relay_result_area = 'RelayList_result_area__2j4GI'
relay_main_info = 'RelayList_main_info__zGpeF'
relay_history = 'RelayList_history_list__13jzg'
relay_pitch = 'RelayList_history_item__UEQst RelayList_type_pitch__3jVsu'
relay_stuff = 'RelayList_stuff__cpnw5'
relay_text = 'RelayList_text__tFNjV'
relay_speed = 'RelayList_speed__xA-Qw'

In [16]:
# web page 열기
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url)

# Optionally wait for the elements to load (adjust the time as needed)
driver.implicitly_wait(10)

In [17]:
try:
    # 요소가 로드될 때까지 기다리기
    wait = WebDriverWait(driver, 10)
    player_areas = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, relay_player_area)))
    
    for player_area in player_areas:
        main_infos = player_area.find_elements(By.CLASS_NAME, relay_main_info)
        for main_info in main_infos:
            main_info_texts = main_info.find_elements(By.CLASS_NAME, relay_text)
            for text_element in main_info_texts:
                print(text_element.text)    

        history_elements = player_area.find_elements(By.CLASS_NAME, relay_history)
        for history in history_elements:
            types = history.find_elements(By.CLASS_NAME, relay_text)
            stuffs = history.find_elements(By.CLASS_NAME, relay_stuff)
            speeds = history.find_elements(By.CLASS_NAME, relay_speed)
            
            for type_element, speed_element, stuff_element in zip(types, speeds, stuffs):
                type_text = type_element.text
                speed_text = int(speed_element.text[:-4])
                stuff_text = stuff_element.text
                print(f'{type_text}: {speed_text}km/h {stuff_text}')
                
        print('\n')
            
except Exception as e:
    print(f'Error: {str(e)}')
finally:
    # 브라우저 닫기
    driver.quit()

김현수 : 중견수 라인드라이브 아웃
타격: 143km/h 직구
볼: 121km/h 체인지업
볼: 143km/h 직구


문성주 : 2루수 땅볼 아웃 (2루수->1루수 송구아웃)
타격: 131km/h 슬라이더
볼: 119km/h 체인지업
볼: 142km/h 직구
파울: 131km/h 슬라이더
스트라이크: 129km/h 슬라이더


홍창기 : 중견수 플라이 아웃
타격: 143km/h 직구
파울: 142km/h 직구
스트라이크: 127km/h 슬라이더
스트라이크: 137km/h 직구
볼: 124km/h 슬라이더
볼: 140km/h 슬라이더
볼: 120km/h 체인지업




In [18]:
# 클래스명 상수 정의
relay_player_area = 'RelayList_player_area__2ur0q'
relay_result_area = 'RelayList_result_area__2j4GI'
relay_main_info = 'RelayList_main_info__zGpeF'
relay_points = 'RelayList_point__1IjBt'
relay_history = 'RelayList_history_list__13jzg'
relay_pitch_num = 'RelayList_pitch_number__YlthP'
relay_pitch = 'RelayList_history_item__UEQst RelayList_type_pitch__3jVsu'
relay_stuff = 'RelayList_stuff__cpnw5'
relay_text = 'RelayList_text__tFNjV'
relay_speed = 'RelayList_speed__xA-Qw'
relay_count = 'RelayList_ball_count__2D8Rd'
relay_change = 'RelayList_history_item__UEQst.Relaylist_type_change__2Zxlf'

In [99]:
def ReadRelay(url):
    # WebDriver 설정 및 브라우저 열기
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    # 웹 페이지 로드
    driver.get(url)

    # 클래스명 상수 정의
    relay_player_area = 'RelayList_player_area__2ur0q'
    relay_result_area = 'RelayList_result_area__2j4GI'
    relay_main_info = 'RelayList_main_info__zGpeF'
    relay_points = 'RelayList_point__1IjBt'
    relay_history = 'RelayList_history_list__13jzg'
    relay_pitch_num = 'RelayList_pitch_number__YlthP'
    relay_pitch = 'RelayList_history_item__UEQst RelayList_type_pitch__3jVsu'
    relay_stuff = 'RelayList_stuff__cpnw5'
    relay_text = 'RelayList_text__tFNjV'
    relay_speed = 'RelayList_speed__xA-Qw'
    relay_count = 'RelayList_ball_count__2D8Rd'
    relay_change = 'RelayList_history_item__UEQst RelayList_type_change__2Zxlf'

    try:
        for num in range(1, 10):
            # 버튼 클릭하기
            button_css_selector = f"#content > div > div.Home_main_section__y9jR4 > section.Home_game_panel__97L_8 > div.Home_game_contents__35IMT > div > div.SetTab_tab_list__1HLl0 > div > div > button:nth-child({num})"

            try:
                button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, button_css_selector)))
                button.click()

                # 정보가 업데이트될 때까지 충분히 기다리기
                time.sleep(2)

                # 업데이트된 정보 가져오기
                page_source = driver.page_source

                # BeautifulSoup로 HTML 파싱
                soup = BeautifulSoup(page_source, 'html.parser')

                print(f'*************** {num}회 ***************\n')

                player_areas = soup.find_all(class_=relay_player_area)
                for player_area in reversed(player_areas):
                    # main_info
                    main_infos = player_area.find_all(class_=relay_main_info)
                    for main_info in main_infos:
                        main_info_texts = main_info.find_all(class_=relay_text)
                        if main_info_texts:
                            for text_element in main_info_texts:
                                print(text_element.get_text())
                        main_info_points = main_info.find_all(class_=relay_points)
                        if main_info_points:
                            for text_element in main_info_points:
                                print(f'(득점) {text_element.get_text()}')
                                
                    # history_list
                    history_elements = player_area.find_all(class_=relay_history)
                    for history in history_elements:
                        pitch_nums = history.find_all(class_=relay_pitch_num)
                        types = history.find_all(class_=relay_text)
                        stuffs = history.find_all(class_=relay_stuff)
                        speeds = history.find_all(class_=relay_speed)
                        counts = history.find_all(class_=relay_count)
                        changes = history.find_all(class_=relay_change)

                        max_length = max(len(pitch_nums), len(types), len(stuffs), len(speeds), len(counts))

                        # 리스트 길이 맞추기
                        while len(pitch_nums) < max_length:
                            pitch_nums.insert(0, None)
                        while len(types) < max_length:
                            types.insert(0, None)
                        while len(stuffs) < max_length:
                            stuffs.insert(0, None)
                        while len(speeds) < max_length:
                            speeds.insert(0, None)
                        while len(counts) < max_length:
                            counts.insert(0, None)  # 공백 문자열이 아닌 빈 항목 추가

                        for pitch_num_element, type_element, speed_element, stuff_element, count_element in zip(reversed(pitch_nums), reversed(types), reversed(speeds), reversed(stuffs), reversed(counts)):
                            pitch_num_text = int(pitch_num_element.get_text()[:-1]) if pitch_num_element else 'N/A'
                            type_text = type_element.get_text() if type_element else 'N/A'
                            speed_text = int(speed_element.get_text()[:-4]) if speed_element else 'N/A'
                            stuff_text = stuff_element.get_text() if stuff_element else 'N/A'
                            count_text = count_element.get_text()[4:] if count_element else 'N/A'
                            print(f'{pitch_num_text}구 {type_text}: {speed_text}km/h {stuff_text} | {count_text}')


                        # changes 처리
                        if changes:
                            print('\n')
                            for change_element in changes:
                                change_text = change_element.get_text()
                                # Add space after '교체' if not followed by a space
                                change_text = re.sub(r'교체(?! )', '교체 ', change_text)
                                # Add space around 'OUT'
                                change_text = re.sub(r'OUT', ' OUT ', change_text)
                                # Add space before 'IN'
                                change_text = re.sub(r'IN', ' IN', change_text)
                                # Add space after '{pos1}:' and '{pos2}:'
                                change_text = re.sub(r'(\w+):', r'\1 :', change_text)
                                print(f'{change_text} ')

                    print('\n')

            except TimeoutException:
                print(f"버튼을 찾을 수 없거나 클릭할 수 없습니다: {button_css_selector}")
                break
            except NoSuchElementException:
                print(f"버튼이 존재하지 않습니다: {button_css_selector}")
                break
            except Exception as e:
                print(f"Unexpected error: {str(e)}")
                break

    except Exception as e:
        print(f'Error: {str(e)}')
    finally:
        # 브라우저 닫기
        driver.quit()

진행중인 경기 URL 가져오기

https://sports.news.naver.com/kbaseball/index

In [100]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re

In [101]:
def GetUrl(team_name):
      url = "https://sports.news.naver.com/kbaseball/index"

      # 팀명에 따른 tag
      tag = {"삼성": 'SS',    "롯데": 'LT', 
             "SSG": 'SK',     "키움": 'WO', 
             "한화": 'HH',    "두산": 'OB',
             "NC": 'NC',      "엔씨": 'NC',      
             "KIA": 'HT',     "기아": 'HT',
             "KT": 'KT',      "케이티": 'KT',
             "LG": 'LG',      "엘지": 'LG'}

      try:
            # 웹 페이지 가져오기
            response = requests.get(url)
            
            # HTTP 응답 상태 확인
            response.raise_for_status()
            
            # BeautifulSoup을 사용하여 HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 모든 a 태그 찾기
            a_tags = soup.find_all('a')
            
            # 정규표현식을 이용하여 '/game/'으로 시작하는 링크만 출력
            #pattern = re.compile(r'^/game/')
            pattern = re.compile(r'/game/\d{8}[A-Z]{4}\d{5}')
              
            for a_tag in a_tags:
                  href = a_tag.get('href')
                  if href and pattern.match(href) and tag[team_name] in href:
                        return "https://m.sports.naver.com" + href
      except requests.exceptions.RequestException as e:
            print(f"Error fetching page: {e}")
      except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [102]:
def GetCurrentMatch(team_name):
      # 현재 날짜와 시간 가져오기
      now = datetime.now()
      
      # 요일 가져오기 (0: 월요일, 1: 화요일, ..., 6: 일요일)
      weekday_number = now.weekday()

      # 현재 시간의 시와 분 가져오기
      current_hour = now.hour
      current_minute = now.minute
      
      # 만약 정수로 사용하고자 한다면, int() 함수를 사용하여 형 변환 가능
      current_hour_int = int(current_hour)
      current_minute_int = int(current_minute)
      
      # 평일 경기 여부 확인
      if weekday_number < 5:
            if current_hour_int > 18 or (current_hour_int == 18 and current_minute_int >= 30):
                  print("경기가 시작되었습니다.")
            else:
                  print("경기 시작 전입니다.")
      # 주말 경기 여부 확인
      else:
            if current_hour_int >= 17:
                  print("경기가 시작되었습니다.")
            else:
                  print("경기 시작 전입니다.")
      
      url = GetUrl(team_name)
      parts = url.split('/')
      if len(parts) == 6:
            # Remove the last part
            base_url_parts = parts[:-1]
            # Join the parts back together with '/'
            url = '/'.join(base_url_parts)
      return url + '/relay'

https://m.sports.naver.com/game/20240619SKSS02024/relay

In [103]:
url = GetCurrentMatch('롯데')
url = 'https://m.sports.naver.com/game/20240620SKSS02024/relay'
print(url)

경기 시작 전입니다.
https://m.sports.naver.com/game/20240620SKSS02024/relay


In [104]:
print(url)

https://m.sports.naver.com/game/20240620SKSS02024/relay


In [105]:
ReadRelay(url)

*************** 1회 ***************

최지훈 : 삼진 아웃
1구 스트라이크: 147km/h 직구 | 0 - 1
2구 볼: 147km/h 직구 | 1 - 1
3구 스트라이크: 147km/h 직구 | 1 - 2
4구 파울: 149km/h 직구 | 1 - 2
5구 파울: 140km/h 슬라이더 | 1 - 2
6구 볼: 148km/h 직구 | 2 - 2
7구 파울: 149km/h 직구 | 2 - 2
8구 스트라이크: 128km/h 슬라이더 | N/A


추신수 : 삼진 아웃
1구 파울: 138km/h 슬라이더 | 0 - 1
2구 스트라이크: 132km/h 체인지업 | 0 - 2
3구 볼: 131km/h 체인지업 | 1 - 2
4구 볼: 148km/h 직구 | 2 - 2
5구 헛스윙: 148km/h 직구 | N/A


최정 : 삼진 아웃
1구 헛스윙: 148km/h 직구 | 0 - 1
2구 파울: 125km/h 슬라이더 | 0 - 2
3구 파울: 149km/h 직구 | 0 - 2
4구 스트라이크: 148km/h 직구 | N/A


김지찬 : 2루수 맞고 중전 1루타
1구 볼: 144km/h 직구 | 1 - 0
2구 스트라이크: 132km/h 슬라이더 | 1 - 1
3구 타격: 137km/h 슬라이더 | N/A


이재현 : 2루수 플라이 아웃
1구 볼: 129km/h 슬라이더 | 1 - 0
2구 파울: 129km/h 슬라이더 | 1 - 1
3구 타격: 141km/h 직구 | N/A


구자욱 : 좌익수 플라이 아웃
1구 스트라이크: 132km/h 포크 | 0 - 1
2구 볼: 119km/h 커브 | 1 - 1
3구 볼: 131km/h 포크 | 2 - 1
4구 타격: 139km/h 직구 | N/A


강민호 : 볼넷
1구 볼: 114km/h 커브 | 1 - 0
2구 볼: 129km/h 슬라이더 | 2 - 0
3구 볼: 114km/h 커브 | 3 - 0
4구 스트라이크: 128km/h 슬라이더 | 3 - 1
5구 볼: 125km/h 슬라이더 | 